# **Chapter 9: Unsupervised Learning Techniques**

## **1. Pendahuluan**

[cite_start]Meskipun sebagian besar aplikasi Machine Learning saat ini didasarkan pada *supervised learning*, kenyataannya sebagian besar data yang tersedia di dunia ini tidak memiliki label[cite: 4583]. [cite_start]Seperti yang dikatakan oleh ilmuwan komputer Yann LeCun: jika kecerdasan adalah sebuah kue, maka *unsupervised learning* adalah kuenya, *supervised learning* adalah lapisan gulanya, dan *reinforcement learning* adalah buah ceri di atasnya[cite: 4584].

Dalam bab ini, kita akan menjelajahi teknik-teknik yang memungkinkan sistem untuk mengeksploitasi data tanpa label tersebut. Fokus utama kita adalah pada tiga tugas utama:
* [cite_start]**Clustering (Pengelompokan):** Menyatukan *instance* serupa ke dalam kelompok-kelompok (cluster)[cite: 4600].
* [cite_start]**Anomaly Detection (Deteksi Anomali):** Mempelajari data "normal" untuk kemudian mendeteksi *instance* yang menyimpang[cite: 4603].
* [cite_start]**Density Estimation (Estimasi Kepadatan):** Memperkirakan fungsi kepadatan probabilitas (PDF) dari proses yang menghasilkan dataset tersebut[cite: 4605].

[cite_start]Memahami teknik-teknik ini sangat penting karena label sering kali mahal dan sulit didapat, sementara data mentah tersedia dalam jumlah besar[cite: 4591].

## **2. Clustering dengan K-Means**

### **Konsep Dasar**
[cite_start]Clustering adalah tugas mengidentifikasi *instance* yang serupa dan menugaskannya ke dalam kelompok-kelompok yang disebut cluster[cite: 4613]. Salah satu algoritma yang paling sederhana dan kuat adalah **K-Means**. [cite_start]Algoritma ini mencari pusat dari setiap kelompok (*centroid*) dan menugaskan setiap titik data ke *centroid* terdekatnya[cite: 4699].

[cite_start]Ini menghasilkan apa yang disebut sebagai *Voronoi tessellation*, di mana batas antar cluster merupakan garis yang membagi jarak antar *centroid* secara adil (seperti yang diilustrasikan secara konseptual pada Gambar 9-3 di buku)[cite: 4724].

### **Algoritma Pelatihan**
[cite_start]Algoritma ini bekerja secara iteratif[cite: 4754]:
1.  Menempatkan *centroid* secara acak.
2.  Memberi label pada setiap *instance* berdasarkan *centroid* terdekat.
3.  Memperbarui posisi *centroid* berdasarkan rata-rata *instance* dalam kelompoknya.
4.  Mengulangi proses hingga *centroid* berhenti bergerak.

Mari kita coba implementasikan menggunakan Scikit-Learn pada data sintetis sederhana.

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

# Membuat data sintetis dengan 5 kelompok (blobs)
X, y = make_blobs(n_samples=1000, centers=5, cluster_std=0.60, random_state=42)

# Melatih model K-Means dengan k=5
k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(X)

# Visualisasi Centroid
plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=1, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            s=100, marker='x', c='red', label='Centroids')
plt.title("Hasil Clustering K-Means")
plt.legend()
plt.show()

print("Centroid yang ditemukan:\n", kmeans.cluster_centers_)

**Interpretasi Hasil:**
[cite_start]Output di atas menunjukkan koordinat pusat dari setiap cluster yang berhasil ditemukan oleh algoritma[cite: 4713]. [cite_start]Perlu diperhatikan bahwa kita harus menentukan jumlah cluster ($k$) di awal[cite: 4702]. [cite_start]Jika kita salah menentukan $k$, cluster mungkin akan digabungkan atau terpecah secara tidak alami (seperti pada Gambar 9-7 di buku)[cite: 4889, 4908].

### **Mencari Jumlah Cluster Optimal**
Kita tidak selalu bisa menebak $k$ hanya dengan melihat data. Ada dua metrik utama untuk mengevaluasi model:
1.  [cite_start]**Inertia:** Jumlah kuadrat jarak antara *instance* dan *centroid* terdekatnya[cite: 4816]. [cite_start]Namun, inertia akan terus turun seiring bertambahnya $k$, jadi kita mencari titik "sikut" (*elbow*) pada kurva[cite: 4911, 4918].
2.  **Silhouette Score:** Rata-rata koefisien siluet dari semua *instance*. [cite_start]Skor ini mengukur seberapa dekat setiap titik dengan clusternya sendiri dibandingkan dengan cluster lain[cite: 4935]. [cite_start]Skor yang mendekati +1 menunjukkan titik tersebut berada jauh di dalam clusternya, sementara skor mendekati -1 menunjukkan kemungkinan salah klasifikasi[cite: 4939].

## **3. Aplikasi Clustering**

Clustering bukan hanya soal mengelompokkan data; ia adalah alat serbaguna yang bisa digunakan untuk berbagai tugas:

* [cite_start]**Segmentasi Gambar:** Mengelompokkan piksel berdasarkan kemiripan warna untuk mengurangi kompleksitas gambar (Gambar 9-12)[cite: 5044, 5081].
* **Preprocessing:** Menggunakan jarak ke *centroid* sebagai fitur baru untuk algoritma *supervised learning*. [cite_start]Ini sering kali meningkatkan performa model (seperti pada contoh dataset digit di buku)[cite: 5084, 5110].
* [cite_start]**Semi-supervised Learning:** Jika label sedikit, kita bisa mengelompokkan data dan menyebarkan label dari *instance* representatif ke seluruh cluster tersebut (*label propagation*)[cite: 5129, 5171].

## **4. DBSCAN**

[cite_start]Berbeda dengan K-Means yang mencari bentuk sferis, **DBSCAN** (*Density-Based Spatial Clustering of Applications with Noise*) mendefinisikan cluster sebagai wilayah dengan kepadatan tinggi[cite: 5209].

**Cara Kerja:**
* [cite_start]Untuk setiap titik, hitung berapa banyak titik lain dalam radius $\epsilon$ (epsilon)[cite: 5210].
* [cite_start]Jika jumlahnya $\geq$ *min_samples*, titik tersebut adalah *core instance*[cite: 5212].
* [cite_start]Semua titik di lingkungan *core instance* masuk ke cluster yang sama[cite: 5214].
* [cite_start]Titik yang tidak memiliki cukup tetangga dan bukan bagian dari lingkungan manapun dianggap sebagai **anomali**[cite: 5217].

[cite_start]Algoritma ini sangat bagus untuk menemukan cluster dengan bentuk yang tidak beraturan (Gambar 9-14)[cite: 5207, 5265].

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

# Membuat data berbentuk bulan sabit (non-sferis)
X_moons, _ = make_moons(n_samples=500, noise=0.05, random_state=42)

# Menggunakan DBSCAN
dbscan = DBSCAN(eps=0.2, min_samples=5)
dbscan.fit(X_moons)

# Visualisasi (Warna -1 adalah anomali/noise)
plt.scatter(X_moons[:, 0], X_moons[:, 1], c=dbscan.labels_, cmap='plasma')
plt.title("DBSCAN pada Moons Dataset")
plt.show()

## **5. Gaussian Mixtures (GMM)**

[cite_start]Gaussian Mixture Model adalah model probabilistik yang mengasumsikan bahwa data dihasilkan dari campuran beberapa distribusi Gaussian dengan parameter yang tidak diketahui[cite: 4392].

[cite_start]Berbeda dengan K-Means yang melakukan *hard clustering* (satu titik tepat di satu cluster), GMM memungkinkan *soft clustering*[cite: 4425]. [cite_start]Ini berarti kita bisa mendapatkan probabilitas bahwa sebuah titik berasal dari cluster tertentu[cite: 4432].

[cite_start]GMM menggunakan algoritma **Expectation-Maximization (EM)** yang bekerja mirip K-Means, tetapi tidak hanya mencari pusat cluster, melainkan juga bentuk, ukuran, dan orientasi elipsnya (Gambar 9-17)[cite: 4423, 4439].

### **Anomaly Detection dengan GMM**
GMM juga sangat efisien untuk deteksi anomali. Kita bisa menghitung kepadatan probabilitas di lokasi tertentu menggunakan `score_samples()`. [cite_start]*Instance* yang berada di wilayah dengan kepadatan sangat rendah kemungkinan besar adalah anomali[cite: 4453, 4459].

In [ ]:
from sklearn.mixture import GaussianMixture

# Melatih GMM
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

# Menghitung skor kepadatan (log-likelihood)
densities = gm.score_samples(X)
threshold = np.percentile(densities, 4) # Mengambil 4% terendah sebagai anomali
anomalies = X[densities < threshold]

plt.scatter(X[:, 0], X[:, 1], c='blue', s=1)
plt.scatter(anomalies[:, 0], anomalies[:, 1], c='red', marker='*', label='Anomalies')
plt.title("Deteksi Anomali dengan GMM")
plt.legend()
plt.show()

## **6. Kesimpulan**

Dalam bab ini, kita telah mempelajari berbagai teknik *unsupervised learning* yang fundamental:
1.  [cite_start]**K-Means** adalah algoritma cepat dan populer untuk data yang terkelompok secara jelas, tetapi terbatas pada bentuk sferis[cite: 4761, 5013].
2.  [cite_start]Kita menggunakan **Silhouette Score** dan **Elbow Method** untuk mengatasi kesulitan dalam memilih jumlah cluster[cite: 4918, 4959].
3.  [cite_start]**DBSCAN** sangat kuat untuk data dengan bentuk arbitrer dan secara otomatis mendeteksi *noise*[cite: 5207, 5310].
4.  [cite_start]**Gaussian Mixture Models (GMM)** memberikan pendekatan probabilistik yang lebih fleksibel dan sangat berguna untuk deteksi anomali[cite: 4392, 4453].

Teknik-teknik ini mempersiapkan kita untuk menangani data dunia nyata yang kompleks, di mana label sering kali merupakan kemewahan yang tidak kita miliki. Pemahaman tentang struktur data ini akan sangat membantu saat kita membangun sistem yang lebih canggih seperti *Recommender Systems* atau *Computer Vision*.